# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import zipfile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
with zipfile.ZipFile("Orders.zip","r") as zip_ref:
    zip_ref.extractall("Orders.csv")

In [3]:
orders = pd.read_csv('Orders.csv/Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
#Check aggregated amount_spent for unique customers:

In [5]:
orders_agg = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent':sum})

In [6]:
orders_agg.describe().T

,count,mean,std,min,25%,50%,75%,max
amount_spent,4347.0,2050.01332,8980.103578,0.0,307.43,671.96,1661.195,280206.02


In [7]:
quantile95 = np.quantile(orders_agg['amount_spent'], q=0.95)
quantile75 = np.quantile(orders_agg['amount_spent'], q=0.75)

In [8]:
quantile75

1661.195

In [9]:
quantile95

5774.277999999996

In [10]:
#a.Find customer type by finding the customers for whom the amount_spent is between 75 and 95% (Preferred) 
#and those for whom the amount_spent is above 95%;

In [11]:
def get_customer(x):     
    if x > quantile75 and x <= quantile95:
        return 'Preferred'
    elif x > quantile95:
        return 'VIP'
    else:
        return 'Other'
    
    
orders_agg['customer_type'] = orders_agg['amount_spent'].apply(get_customer)

In [12]:
orders_agg['customer_type'].value_counts()

Other        3260
Preferred     869
VIP           218
Name: customer_type, dtype: int64

In [13]:
orders_agg.head()

,,amount_spent,customer_type
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12347,Iceland,4310.00,Preferred
12348,Finland,1797.24,Preferred
12349,Italy,1757.55,Preferred
12350,Norway,334.40,Other


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [16]:
VIP = orders_agg[orders_agg['customer_type']=='VIP'].reset_index()

In [18]:
VIP['Country'].value_counts()

United Kingdom     178
Germany             11
France               9
Switzerland          3
Spain                2
Japan                2
Portugal             2
EIRE                 2
Australia            1
Finland              1
Norway               1
Denmark              1
Cyprus               1
Singapore            1
Netherlands          1
Channel Islands      1
Sweden               1
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [21]:
VIPPreferred = orders_agg[orders_agg['customer_type']!='Other'].reset_index()

In [22]:
VIPPreferred['Country'].value_counts()

United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Spain                7
Portugal             7
Norway               7
Italy                5
Finland              5
Channel Islands      4
Australia            4
Japan                4
Cyprus               3
Denmark              3
EIRE                 3
Israel               2
Sweden               2
Singapore            1
Lebanon              1
Poland               1
Iceland              1
Greece               1
Netherlands          1
Austria              1
Canada               1
Malta                1
Name: Country, dtype: int64